In [1]:
import os
import openai
import pandas as pd
import numpy as np
openai.api_key =""
# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

In [2]:
# load csv stock data into data frame 
path = 'fundamental_scoring.csv'

def load_data(data_source='remote',data_location=None):
    if data_source == 'remote':
        pass
    elif data_source != 'remote':
        if data_location is not None:
            return pd.read_csv(data_location)
        else:
            raise ValueError('Local data source path found None; must be string full path.')
    else:
        raise ValueError('Invalid data source')
df = load_data(data_source=None,data_location=path)

In [3]:
sym1 = 'ADBL'
df = df[df['symbol']==sym1]
df

,quarter,fiscal_year,symbol,sector,promoter_shares,public_shares,paidup_capital,net_profit,eps,net_worth,npl,base_rate,cd_ratio,roe,price,date,pe_ratio,pb_ratio,gram_number
0,q4,079/080,ADBL,BANKING,51.0,49.0,1.888439e+10,3.099337e+09,20.620000,214.030000,2.48,10.06,81.73,9.630000,258.5,8/30/2023,12.536372,0.827969,315.117785
1,q3,079/080,ADBL,BANKING,51.0,49.0,1.888439e+10,1.041038e+09,7.900000,242.680000,4.20,10.27,87.19,3.260000,258.5,8/30/2023,32.721519,0.938801,207.692971
2,q2,079/080,ADBL,BANKING,51.0,49.0,1.862063e+10,-1.289336e+08,-4.430000,251.630000,4.52,11.06,87.54,-0.380000,258.5,8/30/2023,-58.352144,0.973424,NaN
3,q1,079/080,ADBL,BANKING,51.0,49.0,1.862063e+10,2.209531e+08,4.230000,258.650000,2.38,9.83,88.67,1.640000,258.5,8/30/2023,61.111111,1.000580,156.898100
4,q4,078/079,ADBL,BANKING,51.0,49.0,1.862063e+10,2.690058e+09,17.930000,184.680000,1.68,8.90,89.17,7.930000,258.5,8/30/2023,14.417178,0.714429,272.955178
5,q3,078/079,ADBL,BANKING,51.0,49.0,1.862063e+10,1.905394e+09,16.790000,247.350000,2.32,8.28,92.51,6.790000,258.5,8/30/2023,15.396069,0.956867,305.683899
6,q2,078/079,ADBL,BANKING,51.0,49.0,1.862063e+10,1.422513e+09,19.110000,178.742683,2.08,7.53,95.32,8.547084,258.5,8/30/2023,13.526949,0.691461,277.227136
7,q1,078/079,ADBL,BANKING,51.0,49.0,1.642264e+10,4.853782e+08,14.700000,198.030000,2.08,7.12,88.99,5.960000,258.5,8/30/2023,17.585034,0.766074,255.926596
8,q4,077/078,ADBL,BANKING,51.0,49.0,1.642264e+10,3.694470e+09,22.496199,192.714526,1.84,7.13,73.53,11.462947,258.5,8/30/2023,11.490830,0.745511,312.322344
9,q3,077/078,ADBL,BANKING,51.0,49.0,1.642264e+10,2.073061e+09,12.623189,186.118484,3.08,7.48,79.92,9.034381,258.5,8/30/2023,20.478184,0.719994,229.916723


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   quarter          28 non-null     object 
 1   fiscal_year      28 non-null     object 
 2   symbol           28 non-null     object 
 3   sector           28 non-null     object 
 4   promoter_shares  28 non-null     float64
 5   public_shares    28 non-null     float64
 6   paidup_capital   23 non-null     float64
 7   net_profit       23 non-null     float64
 8   eps              23 non-null     float64
 9   net_worth        21 non-null     float64
 10  npl              24 non-null     float64
 11  base_rate        24 non-null     float64
 12  cd_ratio         24 non-null     float64
 13  roe              21 non-null     float64
 14  price            28 non-null     float64
 15  date             28 non-null     object 
 16  pe_ratio         23 non-null     float64
 17  pb_ratio         2

In [4]:
def clean_data_frame(dataframe,clean_fields=None):
    if clean_fields is None or clean_fields == []:
        raise ValueError('Fields to be cleaned cannot be None or empty List')

    columns_to_check = clean_fields
    # Drop rows where all specified columns have NaN values
    df_cleaned = dataframe.dropna(subset=columns_to_check, how='all')
    return df_cleaned
cleaned_data = clean_data_frame(df,clean_fields=list(df.columns)[6:14])
cleaned_data

,quarter,fiscal_year,symbol,sector,promoter_shares,public_shares,paidup_capital,net_profit,eps,net_worth,npl,base_rate,cd_ratio,roe,price,date,pe_ratio,pb_ratio,gram_number
0,q4,079/080,ADBL,BANKING,51.0,49.0,1.888439e+10,3.099337e+09,20.620000,214.030000,2.48,10.06,81.73,9.630000,258.5,8/30/2023,12.536372,0.827969,315.117785
1,q3,079/080,ADBL,BANKING,51.0,49.0,1.888439e+10,1.041038e+09,7.900000,242.680000,4.20,10.27,87.19,3.260000,258.5,8/30/2023,32.721519,0.938801,207.692971
2,q2,079/080,ADBL,BANKING,51.0,49.0,1.862063e+10,-1.289336e+08,-4.430000,251.630000,4.52,11.06,87.54,-0.380000,258.5,8/30/2023,-58.352144,0.973424,NaN
3,q1,079/080,ADBL,BANKING,51.0,49.0,1.862063e+10,2.209531e+08,4.230000,258.650000,2.38,9.83,88.67,1.640000,258.5,8/30/2023,61.111111,1.000580,156.898100
4,q4,078/079,ADBL,BANKING,51.0,49.0,1.862063e+10,2.690058e+09,17.930000,184.680000,1.68,8.90,89.17,7.930000,258.5,8/30/2023,14.417178,0.714429,272.955178
5,q3,078/079,ADBL,BANKING,51.0,49.0,1.862063e+10,1.905394e+09,16.790000,247.350000,2.32,8.28,92.51,6.790000,258.5,8/30/2023,15.396069,0.956867,305.683899
6,q2,078/079,ADBL,BANKING,51.0,49.0,1.862063e+10,1.422513e+09,19.110000,178.742683,2.08,7.53,95.32,8.547084,258.5,8/30/2023,13.526949,0.691461,277.227136
7,q1,078/079,ADBL,BANKING,51.0,49.0,1.642264e+10,4.853782e+08,14.700000,198.030000,2.08,7.12,88.99,5.960000,258.5,8/30/2023,17.585034,0.766074,255.926596
8,q4,077/078,ADBL,BANKING,51.0,49.0,1.642264e+10,3.694470e+09,22.496199,192.714526,1.84,7.13,73.53,11.462947,258.5,8/30/2023,11.490830,0.745511,312.322344
9,q3,077/078,ADBL,BANKING,51.0,49.0,1.642264e+10,2.073061e+09,12.623189,186.118484,3.08,7.48,79.92,9.034381,258.5,8/30/2023,20.478184,0.719994,229.916723


In [5]:
import datetime
def recent_data(df):
    df[['fiscal_year_start', 'fiscal_year_end']] = df['fiscal_year'].str.split('/', expand=True)
    # Get the current Gregorian year
    current_gregorian_year = datetime.datetime.now().year
    # Calculate the current Nepali year (approximately)
    current_nepali_year = current_gregorian_year - 1943  # 2021 AD corresponds to 2078 BS
    # Convert 'fiscal_year_end' to numeric for filtering
    df['fiscal_year_end'] = pd.to_numeric(df['fiscal_year_end'], errors='coerce')
    # Calculate the current year (assuming the current year is 080)
    current_year = current_nepali_year
    # Filter the data frame to get the last 3 years of data
    last_3_years_data = df[df['fiscal_year_end'].between(current_year - 2, current_year)]
    # Display the last 3 years of data
    #recent_df = last_3_years_data.drop(columns=['fiscal_year_start', 'fiscal_year_end', 'date'])
    tail_data = last_3_years_data.copy().tail(1)
    sector = tail_data.sector.values[0]
    symbol = tail_data.symbol.values[0]
    last_3_years_data = last_3_years_data.drop(['symbol','sector'],axis=1)
    last_3_years_data.sort_values(by='fiscal_year_end', inplace=True,ascending=False)
    latest_fiscal_year = last_3_years_data.fiscal_year.values[0]
    return last_3_years_data,sector,symbol,latest_fiscal_year

cleaned_data ,sector,symbol,latest_fiscal_yr= recent_data(cleaned_data.copy())

In [6]:
#print(cleaned_data,latest_fiscal_yr)
cleaned_data

,quarter,fiscal_year,promoter_shares,public_shares,paidup_capital,net_profit,eps,net_worth,npl,base_rate,cd_ratio,roe,price,date,pe_ratio,pb_ratio,gram_number,fiscal_year_start,fiscal_year_end
0,q4,079/080,51.0,49.0,1.888439e+10,3.099337e+09,20.620000,214.030000,2.48,10.06,81.73,9.630000,258.5,8/30/2023,12.536372,0.827969,315.117785,079,80
1,q3,079/080,51.0,49.0,1.888439e+10,1.041038e+09,7.900000,242.680000,4.20,10.27,87.19,3.260000,258.5,8/30/2023,32.721519,0.938801,207.692971,079,80
2,q2,079/080,51.0,49.0,1.862063e+10,-1.289336e+08,-4.430000,251.630000,4.52,11.06,87.54,-0.380000,258.5,8/30/2023,-58.352144,0.973424,NaN,079,80
3,q1,079/080,51.0,49.0,1.862063e+10,2.209531e+08,4.230000,258.650000,2.38,9.83,88.67,1.640000,258.5,8/30/2023,61.111111,1.000580,156.898100,079,80
4,q4,078/079,51.0,49.0,1.862063e+10,2.690058e+09,17.930000,184.680000,1.68,8.90,89.17,7.930000,258.5,8/30/2023,14.417178,0.714429,272.955178,078,79
5,q3,078/079,51.0,49.0,1.862063e+10,1.905394e+09,16.790000,247.350000,2.32,8.28,92.51,6.790000,258.5,8/30/2023,15.396069,0.956867,305.683899,078,79
6,q2,078/079,51.0,49.0,1.862063e+10,1.422513e+09,19.110000,178.742683,2.08,7.53,95.32,8.547084,258.5,8/30/2023,13.526949,0.691461,277.227136,078,79
7,q1,078/079,51.0,49.0,1.642264e+10,4.853782e+08,14.700000,198.030000,2.08,7.12,88.99,5.960000,258.5,8/30/2023,17.585034,0.766074,255.926596,078,79
8,q4,077/078,51.0,49.0,1.642264e+10,3.694470e+09,22.496199,192.714526,1.84,7.13,73.53,11.462947,258.5,8/30/2023,11.490830,0.745511,312.322344,077,78
9,q3,077/078,51.0,49.0,1.642264e+10,2.073061e+09,12.623189,186.118484,3.08,7.48,79.92,9.034381,258.5,8/30/2023,20.478184,0.719994,229.916723,077,78


In [7]:
# Find the unique fiscal years and sort them in descending order
unique_years = cleaned_data['fiscal_year'].unique()
unique_years = sorted(unique_years, reverse=True)

# Get the first, second, and third latest fiscal years
latest_fiscal_year = unique_years[0]
second_latest_fiscal_year = unique_years[1]
third_latest_fiscal_year = unique_years[2]
print("First Latest Fiscal Year:", latest_fiscal_year)
print("Second Latest Fiscal Year:", second_latest_fiscal_year)
print("Third Latest Fiscal Year:", third_latest_fiscal_year)


First Latest Fiscal Year: 079/080
Second Latest Fiscal Year: 078/079
Third Latest Fiscal Year: 077/078


In [8]:
latest_year = cleaned_data['fiscal_year_end'].sort_values(ascending=False).iloc[0]
print(latest_year)

80


In [9]:
matching_quarters = cleaned_data[cleaned_data['fiscal_year_end'] == latest_year]['quarter'].sort_values(ascending=True)
matching_quarters = matching_quarters.to_string(index=False)
print(matching_quarters)

q1
q2
q3
q4


In [10]:
class Score:
    def __init__(self,df):
        self.df = df
    def calculate_pe_ratio_score(self,row):
        if row['pe_ratio'] < 15:
            return 10
        elif row['pe_ratio'] > 15 and row['pe_ratio'] < 20:
            return 8
        elif row['pe_ratio'] > 20 and row['pe_ratio'] < 30:
            return 6
        elif row['pe_ratio'] > 15 and row['pe_ratio'] < 20:
            return 5
        else:
            return 4

    def calculate_paid_up_capital_ratio_score(self,row):
        if row['paidup_capital'] < 5e+8:
            return 10
        elif row['paidup_capital'] >= 5e+8 and row['paidup_capital'] < 1e+9:
            return 8
        elif row['paidup_capital'] >= 1e+9 and row['paidup_capital'] < 2e+9:
            return 7
        elif row['paidup_capital'] >= 2e+9  and row['paidup_capital'] < 4e+9:
            return 6
        else:
            return 5

    def calculate_pb_ratio_score(self,row):
        if row['pb_ratio'] < 2.5:
            return 10
        elif row['pb_ratio'] > 2.5 and row['pb_ratio'] < 5:
            return 8
        elif row['pb_ratio'] > 5 and row['pb_ratio'] <= 10:
            return 6
        else:
            return 5

    # def calculate_npl_score(self,row):
    #     if row['pb_ratio'] < 1:
    #         return 10
    #     elif row['pb_ratio'] > 2 and row['pb_ratio'] < 3:
    #         return 8
    #     elif row['pb_ratio'] > 3 and row['pb_ratio'] < 4:
    #         return 6
    #     elif row['pb_ratio'] > 4 and row['pb_ratio'] < 5:
    #         return 5
    #     else:
    #         return 0

    def calculate_gram_number_score(self,row):
        if row['gram_number'] > row['price']:
            return 10
        elif row['gram_number'] > 1 and row['gram_number'] < 50:
            return 8
        elif row['gram_number'] > 50 and row['gram_number'] < 100:
            return 6
        elif row['gram_number'] > 100 and row['gram_number'] < 300:
            return 5
        else:
            return 3
        
    def score_df(self):
        
        # Apply the function to create the 'eps_score' column
        self.df['pe_ratio_score'] = self.df.apply(self.calculate_pe_ratio_score, axis=1)
        
        # Apply the function to create the 'eps_score' column
        self.df['pb_ratio_score'] = self.df.apply(self.calculate_pb_ratio_score, axis=1)
        
        # Apply the function to create the 'eps_score' column
        # self.df['npl_score'] = self.df.apply(self.calculate_npl_score, axis=1)
        
        # Apply the function to create the 'eps_score' column
        self.df['gram_number_score'] = self.df.apply(self.calculate_gram_number_score, axis=1)
        
        # apply for paid up capital score  calculate_paid_up_capital_ratio_score(self,row)
        self.df['paidup_capital_score'] = self.df.apply(self.calculate_paid_up_capital_ratio_score, axis=1)
        return self.df
obj = Score(cleaned_data)
scored_df = obj.score_df()
scored_df

,quarter,fiscal_year,promoter_shares,public_shares,paidup_capital,net_profit,eps,net_worth,npl,base_rate,...,date,pe_ratio,pb_ratio,gram_number,fiscal_year_start,fiscal_year_end,pe_ratio_score,pb_ratio_score,gram_number_score,paidup_capital_score
0,q4,079/080,51.0,49.0,1.888439e+10,3.099337e+09,20.620000,214.030000,2.48,10.06,...,8/30/2023,12.536372,0.827969,315.117785,079,80,10,10,10,5
1,q3,079/080,51.0,49.0,1.888439e+10,1.041038e+09,7.900000,242.680000,4.20,10.27,...,8/30/2023,32.721519,0.938801,207.692971,079,80,4,10,5,5
2,q2,079/080,51.0,49.0,1.862063e+10,-1.289336e+08,-4.430000,251.630000,4.52,11.06,...,8/30/2023,-58.352144,0.973424,NaN,079,80,10,10,3,5
3,q1,079/080,51.0,49.0,1.862063e+10,2.209531e+08,4.230000,258.650000,2.38,9.83,...,8/30/2023,61.111111,1.000580,156.898100,079,80,4,10,5,5
4,q4,078/079,51.0,49.0,1.862063e+10,2.690058e+09,17.930000,184.680000,1.68,8.90,...,8/30/2023,14.417178,0.714429,272.955178,078,79,10,10,10,5
5,q3,078/079,51.0,49.0,1.862063e+10,1.905394e+09,16.790000,247.350000,2.32,8.28,...,8/30/2023,15.396069,0.956867,305.683899,078,79,8,10,10,5
6,q2,078/079,51.0,49.0,1.862063e+10,1.422513e+09,19.110000,178.742683,2.08,7.53,...,8/30/2023,13.526949,0.691461,277.227136,078,79,10,10,10,5
7,q1,078/079,51.0,49.0,1.642264e+10,4.853782e+08,14.700000,198.030000,2.08,7.12,...,8/30/2023,17.585034,0.766074,255.926596,078,79,8,10,5,5
8,q4,077/078,51.0,49.0,1.642264e+10,3.694470e+09,22.496199,192.714526,1.84,7.13,...,8/30/2023,11.490830,0.745511,312.322344,077,78,10,10,10,5
9,q3,077/078,51.0,49.0,1.642264e+10,2.073061e+09,12.623189,186.118484,3.08,7.48,...,8/30/2023,20.478184,0.719994,229.916723,077,78,6,10,5,5


In [11]:
indicator_definitions = """ Paid-Up Capital: Paid-up capital refers to the total amount of capital that has been invested in a company by its shareholders through the purchase of shares. It represents the actual funds contributed by shareholders to the company. 
        
    Net Profit: Net profit, also known as net income or earnings, is the amount of money a company has earned after deducting all its expenses, including operating expenses, interest, taxes, and other costs. It is a key measure of a company's profitability. 

    Earnings Per Share (EPS): EPS is a financial metric that represents the portion of a company's profit allocated to each outstanding share of common stock. It is calculated by dividing the net profit by the total number of outstanding shares.

    Net Worth: Net worth, also known as shareholder's equity or book value, is the difference between a company's total assets and its total liabilities. It represents the value of the company's assets that belongs to its shareholders.

    Non-Performing Loans (NPL): NPL is a measure used in the banking and financial industry. It represents loans that are not generating income for the lender because the borrower is not making interest or principal payments as agreed. A high NPL ratio can indicate credit risk. Npl above 5 is strictly prohibited or penalized by central bank of Nepal(NRB).

    Base Rate: Base rate is the minimum interest rate set by a central bank that commercial banks use as a benchmark for lending to consumers and businesses. It impacts the cost of borrowing and can affect a company's profitability.

    Loan-to-Deposit Ratio (CD Ratio): The CD ratio, or loan-to-deposit ratio, measures the percentage of a bank's deposits that are being used to provide loans to customers. A higher CD ratio can indicate that the bank is lending a significant portion of its deposits. CD Ratio above 90 is strictly prohibited by Central bank of Nepal(NRB).

    Return on Equity (ROE): ROE is a profitability ratio that measures how efficiently a company is using its shareholders' equity to generate profit. It is calculated by dividing net profit by shareholders' equity and is often used to assess a company's management effectiveness.
    
    PE Ratio (Price-to-Earnings Ratio): The PE ratio compares the stock's current market price to its earnings per share (EPS). It's a measure of valuation.A high PE ratio might indicate that the stock is overvalued, while a low PE ratio could suggest undervaluation. Analysts use this ratio to gauge investor sentiment. 

    PB Ratio (Price-to-Book Ratio): The PB ratio compares the stock's market price to its book value per share (BVPS), which is a measure of the company's net assets.A PB ratio above 1 suggests that the stock is trading at a premium to its book value, while a ratio below 1 may indicate it's trading at a discount. This can provide insights into the stock's intrinsic value. 

    Gram Number (Assuming this refers to a specific financial metric): Financial metrics like revenue, earnings, or debt levels are often used in stock analysis.Depending on the specific metric represented by "Gram Number," analysts might use it to assess the financial health or performance of the company. 
    
    Sector: In the context of a stock exchange, a sector refers to a specific industry or segment of the economy in which companies operate. Stock exchanges often classify listed companies into different sectors to help investors and traders analyze and compare companies within the same industry. Common sectors include technology, healthcare, finance, energy, and more. Investors may focus on specific sectors based on their investment strategies and preferences.

    Promoter Shares: Promoter shares, also known as promoter's equity or promoter holding, represent the shares of a company's stock that are owned by its promoters or founders. Promoters are the individuals or entities that started the company and have a significant ownership stake in it. Promoter shares typically come from the initial founding and investment in the company. The percentage of promoter shares held by the founders can vary widely and may influence the company's management and decision-making. Promoter shares are usually subject to lock-up periods and restrictions to prevent insider trading.

    Public Shares: Public shares, also known as public float or public equity, refer to the portion of a company's shares that are available for trading by the general public on a stock exchange. These shares are not owned by the company's promoters or insiders but are instead held by individual and institutional investors. Public shares are freely tradable, and their prices are determined by supply and demand in the open market. The percentage of public shares relative to the total shares outstanding can affect a company's liquidity, stock price volatility, and the degree of control retained by the promoters.
    
    pe_ratio_score: Scores between 1 to 10 and depends on pe_ratio value.
    
    pb_ratio_score: Scores between 1 to 10 and depends on pb_ratio value.
    
    gram_number_score: Scores between 1 to 10 depends on price value.
    
    paidup_capital_score: Scores between 1 to 10 depends on paidup capital value.
    
These indicators are crucial for analyzing a company's financial health, profitability, and risk. Investors and analysts use these metrics to make informed decisions about buying or selling stocks. Keep in mind that the interpretation of these indicators can vary depending on the industry and specific circumstances of the company, so it's essential to consider them in the context of the overall financial analysis."""

In [12]:
indicator_relationship_analysis = """

    Risk Assessment: Financial indicators like NPL and debt levels can be used to assess the risk exposure of companies within a sector. Higher NPL ratios in the banking sector, for instance, may indicate higher credit risk compared to other sectors.

    Investor Sentiment: Analysts can use these indicators to gauge investor sentiment within a sector. For example, if a sector has a high EPS and low NPL ratio, it may attract more investor interest and have a positive sentiment.

    Economic and Market Trends: Changes in base rates can impact various sectors differently. Analysts can monitor base rates and assess how they influence borrowing costs and profitability in different sectors. For example, a rise in base rates may negatively affect the real estate sector but benefit the banking sector.

    Sector Rotation Strategies: Behavioral analysts can use these indicators to identify opportunities for sector rotation strategies. For instance, if they observe that a sector's net profit and EPS are consistently improving, it may indicate a favorable trend for investors to rotate into that sector.

    Market Sentiment Analysis: Analysts can also incorporate sentiment analysis and news sentiment data to understand how news and public perception are influencing specific sectors. Positive or negative sentiment can impact investor behavior and stock prices.

    Cyclical vs. Defensive Analysis: Some sectors are considered cyclical (e.g., technology, consumer discretionary) and are more sensitive to economic cycles, while others are defensive (e.g., healthcare, utilities) and tend to perform well in economic downturns. Analysts can use these indicators to assess whether a sector is behaving in line with its typical cycle.

    Valuation Metrics: Analysts can use these indicators as part of their valuation models to determine whether stocks within a sector are overvalued or undervalued. For example, comparing the EPS of a company to the average EPS of its sector can help assess its relative valuation.

In summary, finance behavioral analysts can use these financial indicators to gain insights into the behavior of specific stock sectors, identify trends, assess risk, and make informed investment decisions. These indicators provide valuable quantitative data that, when analyzed in the context of broader economic and market conditions, can help describe the behavior and prospects of different sectors in the stock market.
"""

In [13]:
query = f"""
As a Behavioral Finance Expert and Stock Trading Expert, please summarize the relations, predictions, correlations, Valuation Metrics, scores, and features in the provided data for the latest fiscal year and latest quarter. If the data is incomplete, indicate that.

Analyze the provided data based on Stock Exchange Behavior and principles.

Indicator Definitions:
{indicator_definitions}

Indicator_Relationship_Analysis:
{indicator_relationship_analysis}

Sector:
{sector}

Stock:
{symbol}

Example:
Compare the current fiscal year {latest_fiscal_year} with previous fiscal year {second_latest_fiscal_year} and {third_latest_fiscal_year} and compare current quarters {matching_quarters} with previous quarters {matching_quarters}.

Data:
{scored_df}


Analyze the provided Data and highlight recent quarters {matching_quarters} with their respective values contrasted against older fiscal_year_end and previous quarters.

Summarize the analysis using the context provided in Indicator Definitions and Indicator_Relationship_Analysis for the given symbol, and provide suggestion paragraphs.
"""

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You follow the given query, example, and analyze the following DataFrame with symbol and sector in DataFrame'},
        {'role': 'user', 'content': query}
    ],
    model=GPT_MODEL,
    temperature=0,
)
# Extract the content from the response
response = response['choices'][0]['message']['content']

print(response)  # Print the response

Based on the provided data for the symbol ADBL in the banking sector, we can analyze the relations, predictions, correlations, valuation metrics, scores, and features for the latest fiscal year 079/080 and the latest quarters q1, q2, q3, and q4. 

In terms of financial indicators, we have data for promoter shares, public shares, paid-up capital, net profit, earnings per share (EPS), net worth, non-performing loans (NPL), base rate, loan-to-deposit ratio (CD ratio), return on equity (ROE), PE ratio, PB ratio, gram number, and various scores related to these metrics.

Analyzing the data for the latest fiscal year 079/080, we can see that the paid-up capital increased compared to the previous fiscal years 078/079 and 077/078. The net profit also increased in the latest fiscal year, resulting in a higher EPS. The net worth of the company also showed an upward trend. However, the NPL ratio increased slightly in the latest fiscal year, which could indicate a higher credit risk. The base rate

In [14]:
query = f"""
Context:
{response}\n
Question: Summarize the Context in paragraph with Highlighting key indicators and give the financial best suggestion.
"""

response1 = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'You follow the given query.'},
        {'role': 'user', 'content': query}
    ],
    model=GPT_MODEL,
    temperature=0.1,
)
# this also good on option type and selected task #### best one for final  # GOOD AT LIMITING THE  NUMBER OF TASKS
response1 = response1['choices'][0]['message']['content']
# response =json.loads(response)
print(response1)

The provided data for the symbol ADBL in the banking sector reveals several key indicators and trends. In the latest fiscal year 079/080, the company's paid-up capital, net profit, and net worth showed positive growth, indicating a healthy financial position. However, the increasing non-performing loans (NPL) ratio suggests potential credit risk that needs to be closely monitored. The valuation metrics, such as the price-to-earnings (PE) ratio and price-to-book (PB) ratio, remained relatively stable throughout the fiscal year and quarters. This suggests that the stock may be trading at a fair value relative to its earnings and book value. 

Based on these findings, the best financial suggestion would be to closely monitor the NPL ratio and assess the company's risk exposure and ability to manage credit risks effectively. Additionally, conducting a sector comparison to evaluate the company's performance relative to its competitors and the overall sector would provide valuable insights. 

In [15]:
def format_paragraphs(paragraph):
    import nltk
    from nltk.tokenize import sent_tokenize
    sentences = sent_tokenize(paragraph)
    for sentence in sentences:
        print(sentence)
    return sentences
res_formatted = format_paragraphs(response)
res_formatted

Based on the provided data for the symbol ADBL in the banking sector, we can analyze the relations, predictions, correlations, valuation metrics, scores, and features for the latest fiscal year 079/080 and the latest quarters q1, q2, q3, and q4.
In terms of financial indicators, we have data for promoter shares, public shares, paid-up capital, net profit, earnings per share (EPS), net worth, non-performing loans (NPL), base rate, loan-to-deposit ratio (CD ratio), return on equity (ROE), PE ratio, PB ratio, gram number, and various scores related to these metrics.
Analyzing the data for the latest fiscal year 079/080, we can see that the paid-up capital increased compared to the previous fiscal years 078/079 and 077/078.
The net profit also increased in the latest fiscal year, resulting in a higher EPS.
The net worth of the company also showed an upward trend.
However, the NPL ratio increased slightly in the latest fiscal year, which could indicate a higher credit risk.
The base rate re

['Based on the provided data for the symbol ADBL in the banking sector, we can analyze the relations, predictions, correlations, valuation metrics, scores, and features for the latest fiscal year 079/080 and the latest quarters q1, q2, q3, and q4.',
 'In terms of financial indicators, we have data for promoter shares, public shares, paid-up capital, net profit, earnings per share (EPS), net worth, non-performing loans (NPL), base rate, loan-to-deposit ratio (CD ratio), return on equity (ROE), PE ratio, PB ratio, gram number, and various scores related to these metrics.',
 'Analyzing the data for the latest fiscal year 079/080, we can see that the paid-up capital increased compared to the previous fiscal years 078/079 and 077/078.',
 'The net profit also increased in the latest fiscal year, resulting in a higher EPS.',
 'The net worth of the company also showed an upward trend.',
 'However, the NPL ratio increased slightly in the latest fiscal year, which could indicate a higher credit 

In [23]:
# def language_translator(message,language='Nepali'):

#     query = f""" You are a English to Nepali language Translator.\n
#     Translate {message} into {language} language and return nepali sentence by sentence followed by '\n'.
#     """
#     response_in_nepali = openai.ChatCompletion.create(
#         messages=[
#             {'role': 'system', 'content': 'You follow the given query.'},
#             {'role': 'user', 'content': query}
#         ],
#         model=GPT_MODEL,
#         temperature=0,
#     )
#     response_in_nepali = response_in_nepali['choices'][0]['message']['content']
#     return response_in_nepali

# message_in_nepali = language_translator(res_formatted)
# print(message_in_nepali)

In [22]:
def language_translator(message, language='Nepali'):
    query = f"You are an English to Nepali language Translator. Translate {message} into {language} language and return Nepali sentence by sentence followed by '\\n'."
    
    conversation = [
        {'role': 'system', 'content': 'You follow the given query.'},
        {'role': 'user', 'content': query}
    ]

    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=conversation,
        temperature=0.7,  # You can adjust the temperature to control randomness
        
    )

    response_text = response['choices'][0]['message']['content']
    return response_text

message_in_nepali = language_translator(res_formatted)
print(message_in_nepali)

बैंकिंग क्षेत्रमा प्रतिष्ठान ADBL को लागि प्रदान गरिएको डाटा आधारमा, हामी नवीनतम आर्थिक बर्ष ०७९/०८० र नवीनतम क्वार्टरहरू q1, q2, q3 र q4 का लागि सम्बन्ध, पूर्वानुमान, सम्पर्कहरू, मूल्यांकन माप, अंक, र सुविधाहरूलाई विश्लेषण गर्न सक्छौं।

आर्थिक सूचकहरूको दृष्टिमा, हामीले प्रमोटर शेयर, सार्वजनिक शेयर, भुक्तान गरिएको पूँजी, नेट मुनाफा, प्रति शेयर अर्जित (EPS), नेट मुल्य, गैर-सफल कर्जा (NPL), मूल दर, कर्जा-जम्मा अनुपात (CD अनुपात), मुनाफा प्रति स्वत्व (ROE), PE अनुपात, PB अनुपात, ग्राम संख्या, र यी मापदण्डहरूसँग सम्बन्धित विभिन्न अंकहरूको डाटा छ।

नवीनतम आर्थिक वर्ष ०७९/०८० को डाटालाई विश्लेषण गर्दा, हामीले अर्थिक बर्ष ०७८/०७९ र ०७७/०७८ सँग तुलना गरेपछि पूँजीको वृद्धि देखियो।

नेट मुनाफाले पनि नवीनतम आर्थिक वर्षमा वृद्धि देखाएपछि, यसको परिणामस्वरूप एपिएस बढी भयो।

कम्पनीको नेट मुल्य पनि उच्चारमा देखायो।

तर, नवीनतम आर्थिक वर्षमा एनपिएल अनुपातले सामान्य रूपमा वृद्धि देखाएको छ, जुन उच्च क्रेडिट जोखिमको संकेत हुन सक्छ।

मूल दर सामान्य रूपमा स्थिर रहेको छ।

तत्कालीन क्वार्टर डाटालाई हेर्दा, ह